In [2]:
import pandas as pd

In [6]:
datas = pd.read_json('ngram/datas/bond_0517dirty.json')

In [7]:
datas.isnull().sum()

date    0
doc     0
dtype: int64

In [1]:
# Call Rate Crawling

import requests
from bs4 import BeautifulSoup
import pandas as pd

start_page = 1
end_page = 544

page_number = 1

date_list = []
rate_list = []

while True:
    URL = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}'.format(page_number)
    r = requests.get(URL)
    sp = BeautifulSoup(r.text,'html.parser')
    results = sp.select('tr')[1:]
    for result in results:
        date_list.append(result.select_one('td.date').text.strip())
        rate_list.append(result.select_one('td.num').text.strip())

    page_number += 1
    if page_number > end_page:
        break

call_df = pd.DataFrame(list(zip(date_list,rate_list)))

In [ ]:
call_df.columns = ['date','rate']

In [61]:
call_df

,rate
date,
2020-07-27,0.49
2020-07-24,0.48
2020-07-23,0.46
2020-07-22,0.45
2020-07-21,0.46
...,...
2005-03-30,3.25
2005-03-29,3.24
2005-03-28,3.25


In [7]:
import datetime

In [9]:
call_df['date'] = list(map(lambda i : datetime.datetime.strptime(i, '%Y.%m.%d'), list(call_df['date'])))

In [11]:
start_date = call_df['date'][len(call_df)-1]
end_date = call_df['date'][0]

print(start_date, end_date)


2005-03-24 00:00:00 2020-07-27 00:00:00


5604

In [64]:
call_df

,rate
date,
2020-07-27,0.49
2020-07-24,0.48
2020-07-23,0.46
2020-07-22,0.45
2020-07-21,0.46
...,...
2005-03-30,3.25
2005-03-29,3.24
2005-03-28,3.25


In [62]:
day_range = (end_date - start_date).days

temp = []
for i in range(day_range):
    temp.append((start_date + datetime.timedelta(i)).date())

call_final = pd.DataFrame(temp, columns = ['date'])

In [76]:
call_rate = call_final.set_index('date')
call_rate['rate'] = call_df.rate


In [80]:
call_final = call_rate.sort_values(by='date').fillna(method='pad')

In [84]:
call_final.to_json('ngram/target/call_raw.json')

In [184]:
# 금리 데이터 기간, rate limit 별로 저장

import pandas as pd
import datetime

datas = pd.read_json('ngram/target/call_raw.json')

date_count = 30
rate_limit = 0.03

temp = []
for i in range(len(datas)-date_count):
    rate_change = float(datas['rate'][i+date_count]) - float(datas['rate'][i])
    if rate_change >= rate_limit:
        temp.append('up')
    elif rate_change <= -rate_limit:
        temp.append('down')
    else:
        temp.append(0)

temp
new_data = datas.iloc[:-date_count]
new_data['ud'] = temp
new_data.reset_index(inplace=True)
new_data.columns = ['date', 'rate', 'ud']
new_data['date'] = list(map(lambda i : i.date(), new_data['date']))
new_data.to_json('ngram/target/call_{}_{}.json'.format(str(date_count), str(rate_limit)))

In [154]:
# Read
date_count = 30
rate_limit = 0.02
call_data = pd.read_json('ngram/target/call_{}_{}.json'.format(str(date_count), str(rate_limit)))

,date,rate,ud
40,2005-05-03,3.30,down
43,2005-05-06,3.27,down
44,2005-05-07,3.27,down
45,2005-05-08,3.27,down
47,2005-05-10,3.30,down
...,...,...,...
5559,2020-06-12,0.46,up
5560,2020-06-13,0.46,up
5563,2020-06-16,0.46,up
5567,2020-06-20,0.46,down


In [ ]:
call_date[(call_data['date'] >= datetime.date(2005,5,1))&(call_data['ud']!=0)]

In [182]:
ngram_df = pd.read_json('total_ngram.json')
ngram_df['date'] = list(map(lambda i : i.date(), ngram_df['date']))

In [187]:
rate_df = pd.read_json('ngram/target/call_30_0.03.json')
rate_df['date'] = list(map(lambda i : i.date(), rate_df['date']))

In [188]:
ngram_df.set_index('date')

In [190]:
ngram_df.set_index('date', inplace=True)

In [194]:
ngram_df['ud'] = rate_df.set_index('date')['ud']

In [195]:
ngram_df = ngram_df[['ngram','ud']]

In [207]:
rate_df[rate_df['date'] == datetime.date(2020,7,15)]

,date,rate,ud


In [212]:
range(len(ngram_df)-1, 0)

range(305072, 0)

In [213]:
ngram_df.reset_index(inplace=True)

In [214]:
ngram_df

,date,ngram,ud
0,2005-05-12,"[부진/NNG;글로벌/NNG;경기/NNG;둔화/NNG;우려/NNG, 소비/NNG;설...",0
1,2005-06-09,"[경상/NNG;수지/NNG;적자/NNG;재정/NNG;적자/NNG, 국제/NNG;금융...",down
2,2005-06-23,"[대출/NNG;촉진/NNG, 걸림돌/NNG, 개선/NNG]",0
3,2005-07-07,"[자산/NNG;가격/NNG;상승/NNG;물가/NNG;상승/NNG, 물가/NNG;안정...",up
4,2005-08-11,"[부동산/NNG;가격/NNG;상승/NNG;금리/NNG;인상/NNG, 국채/NNG;수...",0
...,...,...,...
305068,2020-07-15,"[경기/NNG;침체/NNG, 완화/NNG]",NaN
305069,2020-07-15,"[코픽스/NNG;떨어/VV, 돈/NNG;확보/NNG, 하락/NNG, 내림/NNG]",NaN
305070,2020-07-15,"[저축성/NNG;예금/NNG;늘/VV, 경기/NNG;악화/NNG, 통화/NNG;급증...",NaN
305071,2020-07-15,"[최대/NNG, 충족/NNG]",NaN


In [221]:
for i in range(len(ngram_df)):
    if ngram_df['ngram'][i] == []:
        ngram_df.drop(i, axis=0, inplace=True)
ngram_df

,date,ngram,ud
0,2005-05-12,"[부진/NNG;글로벌/NNG;경기/NNG;둔화/NNG;우려/NNG, 소비/NNG;설...",0
1,2005-06-09,"[경상/NNG;수지/NNG;적자/NNG;재정/NNG;적자/NNG, 국제/NNG;금융...",down
2,2005-06-23,"[대출/NNG;촉진/NNG, 걸림돌/NNG, 개선/NNG]",0
3,2005-07-07,"[자산/NNG;가격/NNG;상승/NNG;물가/NNG;상승/NNG, 물가/NNG;안정...",up
4,2005-08-11,"[부동산/NNG;가격/NNG;상승/NNG;금리/NNG;인상/NNG, 국채/NNG;수...",0
...,...,...,...
305067,2020-07-15,"[실망/NNG, 중단/NNG]",NaN
305068,2020-07-15,"[경기/NNG;침체/NNG, 완화/NNG]",NaN
305069,2020-07-15,"[코픽스/NNG;떨어/VV, 돈/NNG;확보/NNG, 하락/NNG, 내림/NNG]",NaN
305070,2020-07-15,"[저축성/NNG;예금/NNG;늘/VV, 경기/NNG;악화/NNG, 통화/NNG;급증...",NaN


In [223]:
ngram_df.dropna(inplace=True)

In [225]:
ngram_df.reset_index()[['date', 'ngram', 'ud']].to_json('train_data.json')

In [227]:
ngram_df

,date,ngram,ud
0,2005-05-12,"[부진/NNG;글로벌/NNG;경기/NNG;둔화/NNG;우려/NNG, 소비/NNG;설...",0
1,2005-06-09,"[경상/NNG;수지/NNG;적자/NNG;재정/NNG;적자/NNG, 국제/NNG;금융...",down
2,2005-06-23,"[대출/NNG;촉진/NNG, 걸림돌/NNG, 개선/NNG]",0
3,2005-07-07,"[자산/NNG;가격/NNG;상승/NNG;물가/NNG;상승/NNG, 물가/NNG;안정...",up
4,2005-08-11,"[부동산/NNG;가격/NNG;상승/NNG;금리/NNG;인상/NNG, 국채/NNG;수...",0
...,...,...,...
304964,2020-06-26,"[매출/NNG;감소/NNG, 피해/NNG, 최대/NNG, 개선/NNG]",0
304965,2020-06-26,"[경기/NNG;판단/NNG;상승/NNG, 경기/NNG;전망/NNG;하락/NNG, 전...",0
304966,2020-06-26,"[소비자/NNG;심리/NNG;긍정적/VAX, 지표/NNG;낮/VA, 하락/NNG, ...",0
304967,2020-06-26,"[주택가격/NNG;전망/NNG;상승/NNG, 취업기회/NNG;전망/NNG;오르/VV...",0


In [197]:
ngram_df.isnull().sum()

ngram      0
ud       603
dtype: int64